In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder , StandardScaler


In [10]:

df = pd.read_csv(r"C:\Users\moham\Documents\GitHub\MLOps-pipline\datasets\Titanic-Dataset.csv")

df = df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", "Survived"]]
df = df.dropna()

df["Sex"] = LabelEncoder().fit_transform(df["Sex"])
df["Embarked"] = LabelEncoder().fit_transform(df["Embarked"])

numerical_cols = ["Age", "SibSp", "Parch", "Fare"]
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

X = df.drop("Survived", axis=1).values
y = df["Survived"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)


In [11]:
class TitanicModel(nn.Module):
    def __init__(self, input_size):
        super(TitanicModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 2)  

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [12]:
model = TitanicModel(input_size=X_train.shape[1])


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 100
for epoch in range(epochs):
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            train_preds = torch.argmax(model(X_train), dim=1)
            train_acc = accuracy_score(y_train, train_preds)

            test_preds = torch.argmax(model(X_test), dim=1)
            test_acc = accuracy_score(y_test, test_preds)

        print(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item():.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")

Epoch 10/100 | Loss: 0.6058 | Train Acc: 0.6854 | Test Acc: 0.6503
Epoch 20/100 | Loss: 0.5428 | Train Acc: 0.7293 | Test Acc: 0.6923
Epoch 30/100 | Loss: 0.4846 | Train Acc: 0.7680 | Test Acc: 0.7832
Epoch 40/100 | Loss: 0.4412 | Train Acc: 0.8049 | Test Acc: 0.8182
Epoch 50/100 | Loss: 0.4203 | Train Acc: 0.8032 | Test Acc: 0.8252
Epoch 60/100 | Loss: 0.4109 | Train Acc: 0.8049 | Test Acc: 0.8252
Epoch 70/100 | Loss: 0.4035 | Train Acc: 0.8155 | Test Acc: 0.8322
Epoch 80/100 | Loss: 0.3990 | Train Acc: 0.8190 | Test Acc: 0.8182
Epoch 90/100 | Loss: 0.3952 | Train Acc: 0.8225 | Test Acc: 0.8182
Epoch 100/100 | Loss: 0.3918 | Train Acc: 0.8243 | Test Acc: 0.8112


In [15]:
import os
dummy_input = torch.randn(1, X_train.shape[1])  

os.makedirs("model", exist_ok=True)
torch.onnx.export(
    model,
    dummy_input,
    "model/titanic_model.onnx",
    input_names=["input"],
    output_names=["output"],
    do_constant_folding=True,
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11
)

print("Exported to model/titanic_model.onnx")


Exported to model/titanic_model.onnx
